In [1]:
%matplotlib inline
import dgl
import copy
import glob
import pprint
import numpy as np
import torch
import torch.nn as nn
import awkward as ak
import networkx as nx
import matplotlib.pyplot as plt
from os import path
from pathlib import Path
from trainresults import TrainResults
from train_eval_func import train, evaluate
from copy import deepcopy
from dgl.data import DGLDataset
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from TauGraphDatasetInfo import TauGraphDatasetInfo
from MPGNN import MPGNN, MPGNN_Mean
from TauGraphDataset import TauGraphDataset, GetNodeFeatureVectors, GetEdgeFeatureVectors, GetNeighborNodes, GetEdgeList

plt.rcParams.update({'font.size': 20})
plt.rcParams['text.usetex'] = True
lw = 2
xyLabelFontSize = 20
xLabelPad = 10
yLabelPad = 15
pp = pprint.PrettyPrinter()

Using backend: pytorch


In [2]:
# The evaluation function
@torch.no_grad()
def evaluate(model, device, dataloader, loss_fn):
    model = model.to(device)
    model.eval()
    y_true = []
    y_logits = []
    epochLoss = 0.0
    batchIter = 0

    for batched_graph, labels in dataloader:
        batched_graph = batched_graph.to(device)
        labels = labels.to(device)
        nodeFeatVec = GetNodeFeatureVectors(batched_graph)
        edgeFeatVec = GetEdgeFeatureVectors(batched_graph)

        with torch.no_grad():
            pred = model(batched_graph, nodeFeatVec, edgeFeatVec)

        loss = loss_fn(pred,labels)
        epochLoss += loss.item()
        y_true.append(labels.detach().cpu())
        y_logits.append(pred.detach().cpu())
        batchIter += 1

    y_true = torch.cat(y_true, dim = 0).numpy()
    y_logits = torch.cat(y_logits, dim = 0)
    y_softmax = nn.functional.softmax(y_logits, dim=1)
    y_scoreClass1 = y_softmax[:, 1]
    y_pred = y_logits.numpy().argmax(1)
    
    num_correct_pred = (y_pred == y_true).sum().item()
    num_total_pred = len(y_true)
    acc =  num_correct_pred / num_total_pred
    
    evalDict = {
        "y_true": y_true.tolist(), 
        "y_logits": y_logits.tolist(), 
        "y_scoreClass1": y_scoreClass1.tolist(),
        "y_pred": y_pred.tolist(), 
        "acc": acc,
        "loss" : epochLoss / batchIter
    }

    return evalDict


def train(model, device, dataloader, optimizer, loss_fn, batchsize, results):
    model = model.to(device)
    model.train()
    
    epochLoss = 0.0
    batchIter = 0
    
    for batched_graph, labels in dataloader:
        batched_graph = batched_graph.to(device)
        labels = labels.to(device)
        nodeFeatVec = GetNodeFeatureVectors(batched_graph)
        edgeFeatVec = GetEdgeFeatureVectors(batched_graph)

        #forward
        pred =  model(batched_graph, nodeFeatVec, edgeFeatVec)

        # compute loss
        loss = loss_fn(pred, labels)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # multiply running loss by the number of graphs, 
        # since CrossEntropy loss calculates mean of the losses of the graphs in the batch
        runningTotalLoss = loss.item() #* batchsize
        results.addRunningLoss(runningTotalLoss)
        epochLoss += runningTotalLoss
        batchIter += 1
        
    return epochLoss/batchIter

In [3]:
# training
def trainEpochs(model, device, dataloader, optimizer, loss_fn, batchsize, nEpochs):
    results = TrainResults()
    results.startTrainingTimer()
    bestModel = None
    bestValAcc = 0.0

    for epoch in range(nEpochs):
        # train
        epochLoss = train(model, device, dataloader, optimizer, loss_fn, batchsize, results)

        # evaluate
        train_result = evaluate(model, device, train_dataloader, loss_fn)
        val_result = evaluate(model, device, val_dataloader, loss_fn)
        test_result = evaluate(model, device, test_dataloader, loss_fn)

        results.addEpochResult(epochLoss, train_result, val_result, test_result)
        results.printLastResult()

        if results.best_val_acc > bestValAcc:
            bestValAcc = results.best_val_acc
            bestModel = copy.deepcopy(model)
    
    results.endTrainingTimer()

    return results, bestModel

In [4]:
def getDatasetNames(datasetDir):
    files = glob.glob(datasetDir + '/*.json', recursive=True)
    files.sort()
    datasetDirectories = [path.dirname(file) for file in files]
    datasetnames = [path.normpath(dir).split(path.sep)[-1] for dir in datasetDirectories]
    return datasetDirectories, datasetnames

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

datasetDir = '/ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5_THESIS/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets'
datasetDirs, datasetNames = getDatasetNames(datasetDir)
print(datasetDirs)
print(datasetNames)

device: cpu
['/ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5_THESIS/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets']
['Graphs_DYJetsToLL_M-50_genuineTaus_and_jets']


datasetName = datasetNames[0]
datasetDir = datasetDirs[0]
dataset = TauGraphDataset(datasetName, datasetDir)
dataset.printProperties()

graph, label = dataset[0]
print(graph)
print(f'Label: {label}')
print(GetNodeFeatureVectors(graph))

In [8]:
import time
now = time.time()

paramList = [(16,128, 2)] # node_hiddenfeats, #edge_hiddenfeats, #num_messagepasses
            

batchSize = 1024
print(f'Device: {device}')

for n_hidden,e_hidden, msgpasses in paramList:
    for i in range(len(datasetDirs)):
        dataset = TauGraphDataset(datasetNames[i], datasetDirs[i])
        splitIndices = dataset.get_split_indices()

        train_sampler = SubsetRandomSampler(splitIndices['train'])
        val_sampler = SubsetRandomSampler(splitIndices['valid'])
        test_sampler = SubsetRandomSampler(splitIndices['test'])

        train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=batchSize, drop_last=False)
        val_dataloader = GraphDataLoader(dataset, sampler=val_sampler, batch_size=batchSize, drop_last=False)
        test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=batchSize, drop_last=False)

        # Create the model with given dimensions
        model = MPGNN(
            dataset.dim_nfeats, dataset.dim_efeats, 
            node_out_feats=n_hidden, edge_hidden_feats=e_hidden, 
            num_step_message_passing=msgpasses, n_classes=dataset.num_graph_classes).to(device)

        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        loss_fn = nn.CrossEntropyLoss()
        epochs = 30

        # train
        print(f'Beginning training on dataset {datasetNames[i]}')
        results, bestmodel = trainEpochs(model, device, train_dataloader, optimizer, loss_fn, batchSize, epochs)
        results.printBestResult()

        # save results
        outputFolder = path.join(datasetDirs[i], f'Output_MPGNN_NHiddenFeats_{n_hidden}_EHiddenFeats_{e_hidden}_MsgPasses_{msgpasses}')
        Path(outputFolder).mkdir(parents=True, exist_ok=True)

        results.savePlots(outputFolder)
        results.dumpSummary(outputFolder)
        results.pickledump(outputFolder)

        # save the best model for inference. (when loading for inference -> model.eval()!! )
        # https://pytorch.org/tutorials/beginner/saving_loading_models.html#what-is-a-state-dict
        torch.save(bestmodel.state_dict(), path.join(outputFolder, 'model.pt'))

        print(f'------------------({i+1}/{len(datasetDirs)}) models trained------------------\n')


end = time.time()
elapsed = end - now
print(f'{elapsed} seconds elapsed')

Device: cpu
Done loading data from cached files.
Beginning training on dataset Graphs_DYJetsToLL_M-50_genuineTaus_and_jets
Epoch: 0, Loss: 0.5151, Validation Loss: 0.4062, Train: 0.850, Validation: 0.852, Test: 0.855, AUC: 0.951
Epoch: 1, Loss: 0.3804, Validation Loss: 0.3564, Train: 0.897, Validation: 0.897, Test: 0.900, AUC: 0.952
Epoch: 2, Loss: 0.3403, Validation Loss: 0.3380, Train: 0.873, Validation: 0.873, Test: 0.876, AUC: 0.952
Epoch: 3, Loss: 0.3153, Validation Loss: 0.2947, Train: 0.901, Validation: 0.901, Test: 0.905, AUC: 0.958
Epoch: 4, Loss: 0.2774, Validation Loss: 0.2906, Train: 0.883, Validation: 0.882, Test: 0.883, AUC: 0.958
Epoch: 5, Loss: 0.2692, Validation Loss: 0.2538, Train: 0.904, Validation: 0.904, Test: 0.906, AUC: 0.961
Epoch: 6, Loss: 0.2553, Validation Loss: 0.2565, Train: 0.908, Validation: 0.907, Test: 0.911, AUC: 0.963
Epoch: 7, Loss: 0.2487, Validation Loss: 0.2463, Train: 0.904, Validation: 0.904, Test: 0.906, AUC: 0.963
Epoch: 8, Loss: 0.2444, Valid

In [ ]:
import time
now = time.time()

paramList = [(16,64, 2),# node_hiddenfeats, #edge_hiddenfeats, #num_messagepasses
          (16,128, 2),
          (32, 8, 2),
          (32, 16, 2),
          (32, 32, 2),
          (16, 8, 7)]

batchSize = 1024
print(f'Device: {device}')

for n_hidden,e_hidden, msgpasses in paramList:
    for i in range(len(datasetDirs)):
        dataset = TauGraphDataset(datasetNames[i], datasetDirs[i])
        splitIndices = dataset.get_split_indices()

        train_sampler = SubsetRandomSampler(splitIndices['train'])
        val_sampler = SubsetRandomSampler(splitIndices['valid'])
        test_sampler = SubsetRandomSampler(splitIndices['test'])

        train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=batchSize, drop_last=False)
        val_dataloader = GraphDataLoader(dataset, sampler=val_sampler, batch_size=batchSize, drop_last=False)
        test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=batchSize, drop_last=False)

        # Create the model with given dimensions
        model = MPGNN_Mean(
            dataset.dim_nfeats, dataset.dim_efeats, 
            node_out_feats=n_hidden, edge_hidden_feats=e_hidden, 
            num_step_message_passing=msgpasses, n_classes=dataset.num_graph_classes).to(device)

        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        loss_fn = nn.CrossEntropyLoss()
        epochs = 30

        # train
        print(f'Beginning training on dataset {datasetNames[i]}')
        results, bestmodel = trainEpochs(model, device, train_dataloader, optimizer, loss_fn, batchSize, epochs)
        results.printBestResult()

        # save results
        outputFolder = path.join(datasetDirs[i], f'Output_MPGNN_Mean_NHiddenFeats_{n_hidden}_EHiddenFeats_{e_hidden}_MsgPasses_{msgpasses}')
        Path(outputFolder).mkdir(parents=True, exist_ok=True)

        results.savePlots(outputFolder)
        results.dumpSummary(outputFolder)
        results.pickledump(outputFolder)

        # save the best model for inference. (when loading for inference -> model.eval()!! )
        # https://pytorch.org/tutorials/beginner/saving_loading_models.html#what-is-a-state-dict
        torch.save(bestmodel.state_dict(), path.join(outputFolder, 'model.pt'))

        print(f'------------------({i+1}/{len(datasetDirs)}) models trained------------------\n')


end = time.time()
elapsed = end - now
print(f'{elapsed} seconds elapsed')

Device: cpu
Done loading data from cached files.
Beginning training on dataset Graphs_DYJetsToLL_M-50_genuineTaus_and_jets
Epoch: 0, Loss: 0.5858, Validation Loss: 0.4950, Train: 0.791, Validation: 0.790, Test: 0.792, AUC: 0.873
Epoch: 1, Loss: 0.4366, Validation Loss: 0.3924, Train: 0.832, Validation: 0.831, Test: 0.833, AUC: 0.921
Epoch: 2, Loss: 0.3737, Validation Loss: 0.3491, Train: 0.872, Validation: 0.870, Test: 0.873, AUC: 0.938
Epoch: 3, Loss: 0.3456, Validation Loss: 0.3324, Train: 0.872, Validation: 0.872, Test: 0.875, AUC: 0.943


In [ ]:
import torch
model = MPGNN(
    7, 3, 
    node_out_feats=16, edge_hidden_feats=16, 
    num_step_message_passing=2, n_classes=2)

folder = '/ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5_THESIS/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets/Output_MPGNN/model.pt'
model.load_state_dict(torch.load(folder))
model.eval()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batchSize=64
dataset = TauGraphDataset(datasetNames[0], datasetDirs[0])
splitIndices = dataset.get_split_indices()
test_sampler = SubsetRandomSampler(splitIndices['test'])
test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=batchSize, drop_last=False)

res = evaluate(model, device, test_dataloader)

In [ ]:
print(res['y_true'][:10])
print(res['y_logits'][:10])
print(res['y_scoreClass1'][:10])

In [ ]:
"""
    y_true = torch.cat(y_true, dim = 0).numpy()
    y_logits = torch.cat(y_logits, dim = 0)
    y_softmax = nn.functional.softmax(y_logits, dim=1)
    y_scoreClass1 = y_softmax[:, 1]
    y_pred = y_logits.numpy().argmax(1)
"""
import torch
logits = torch.tensor(res["y_logits"])
print("list to tensor done")

# NN output plot
predictions = torch.nn.functional.softmax(logits, dim=1).numpy()
print("softmax done")
print(f'len(predictions): {len(predictions)}')
#print(predictions)

# TODO: check which order is actually signal (genuineTau) and which are background (fakeTau)
genuineTau_decisions = predictions[:, 0]
fakeTau_decisions = predictions[:, 1]

plt.figure(figsize=(9,7))

plt.hist(genuineTau_decisions, label='Genuine Taus', 
        histtype='step', # lineplot that's unfilled
        linewidth=lw)
plt.hist(fakeTau_decisions, label='Jets', 
        histtype='step', # lineplot that's unfilled
        linewidth=lw)
plt.xlabel('Neural Network output') # add x-axis label
plt.ylabel('Frequency') # add y-axis label
plt.legend(loc="upper center") # add legend
plt.show()

In [ ]:
print(predictions[:100, 0])
print(predictions[:100, 1])